In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath(''))

# Add the PythonPrep directory to the path
pythonprep_dir = os.path.join(os.path.dirname(notebook_dir), 'PythonPrep') if 'PythonPrep' not in notebook_dir else notebook_dir
sys.path.append(pythonprep_dir)

from paths import main_path
import warnings
import sys
import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath(''))

# Add the PythonPrep directory to the path
pythonprep_dir = os.path.join(os.path.dirname(notebook_dir), 'PythonPrep') if 'PythonPrep' not in notebook_dir else notebook_dir
sys.path.append(pythonprep_dir)

from paths import main_path

warnings.filterwarnings("ignore")

In [ ]:
path_input = main_path + "/Democracy/Democracy_Main/MainAnalysis/input"
df = pd.read_csv(f"{path_input}/channels21st/new_channels/proportion-of-important-sites-for-freshwater-biodiversity-covered-by-protected-areas.csv")
df


,Entity,Code,Year,15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR
0,Afghanistan,AFG,2000,0.00
1,Afghanistan,AFG,2001,0.00
2,Afghanistan,AFG,2002,0.00
3,Afghanistan,AFG,2003,0.00
4,Afghanistan,AFG,2004,0.00
...,...,...,...,...
4020,Zimbabwe,ZWE,2018,82.01
4021,Zimbabwe,ZWE,2019,82.01
4022,Zimbabwe,ZWE,2020,82.01
4023,Zimbabwe,ZWE,2021,82.01


In [3]:
df = df[df['Year'] >= 2001]
df

,Entity,Code,Year,15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR
1,Afghanistan,AFG,2001,0.00
2,Afghanistan,AFG,2002,0.00
3,Afghanistan,AFG,2003,0.00
4,Afghanistan,AFG,2004,0.00
5,Afghanistan,AFG,2005,0.00
...,...,...,...,...
4020,Zimbabwe,ZWE,2018,82.01
4021,Zimbabwe,ZWE,2019,82.01
4022,Zimbabwe,ZWE,2020,82.01
4023,Zimbabwe,ZWE,2021,82.01


In [4]:
first_years = df.groupby('Entity')['Year'].min()

countries_starting_in_2001 = first_years[first_years == 2001].index
filtered_df = df[(df['Entity'].isin(countries_starting_in_2001)) & (df['15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR'] != 0)]
filtered_df = filtered_df[filtered_df['Entity'].isin(countries_starting_in_2001)]

In [5]:
df = filtered_df.copy()
def filter_countries_starting_in_2001(dataframe):
    first_years = dataframe.groupby("Entity")["Year"].min().reset_index()
    start_in_2001 = first_years[first_years["Year"] == 2001]["Entity"]
    filtered_df = dataframe[dataframe["Entity"].isin(start_in_2001)]
    return filtered_df

filtered_df = filter_countries_starting_in_2001(df)

In [6]:
filtered_df

,Entity,Code,Year,15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR
24,Algeria,DZA,2001,32.08
25,Algeria,DZA,2002,32.08
26,Algeria,DZA,2003,39.88
27,Algeria,DZA,2004,69.16
28,Algeria,DZA,2005,69.16
...,...,...,...,...
4020,Zimbabwe,ZWE,2018,82.01
4021,Zimbabwe,ZWE,2019,82.01
4022,Zimbabwe,ZWE,2020,82.01
4023,Zimbabwe,ZWE,2021,82.01


In [7]:
filtered_df.Entity.nunique()

149

In [8]:
df = filtered_df.copy()

In [9]:
df['growth_rate'] = df.groupby('Entity')['15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR'].pct_change() * 100



In [10]:
 periods = {
    '2001-2019': (2001, 2019),
    #'2019-2022': (2019, 2022),
}

result_df = pd.DataFrame()

for period, (start_year, end_year) in periods.items():

    period_df = df[(df['Year'] >= start_year) & (df['Year'] <= end_year)]
    avg_life_expectancy = period_df.groupby('Entity')['growth_rate'].mean().rename(period)
    
    result_df = pd.concat([result_df, avg_life_expectancy], axis=1)

result_df.reset_index(inplace=True)
result_df['country'] = result_df['index']
result_df = result_df.drop('index', axis=1)
result_df = result_df[['country'] + [col for col in result_df.columns if col != 'country']]

result_df.rename(columns={
    '2001-2019': 'avg_water_2001_2019',
    #'2019-2022': 'avg_water_2019_2022',
}, inplace=True)

result_df

,country,avg_water_2001_2019
0,Algeria,6.194113
1,Angola,0.000000
2,Argentina,1.994157
3,Armenia,1.177986
4,Australia,2.911199
...,...,...
144,Venezuela,0.000000
145,Vietnam,7.847490
146,World,2.292080
147,Zambia,0.000000


In [11]:
result_df[result_df['avg_water_2001_2019']==np.inf]

,country,avg_water_2001_2019


In [ ]:
###

In [12]:
df = pd.read_csv('/Users/leonardofancello/Desktop/Yale/Democracy/MainAnalysisRep/input/channels21st/new_channels/proportion-of-important-sites-for-freshwater-biodiversity-covered-by-protected-areas.csv')
df


,Entity,Code,Year,15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR
0,Afghanistan,AFG,2000,0.00
1,Afghanistan,AFG,2001,0.00
2,Afghanistan,AFG,2002,0.00
3,Afghanistan,AFG,2003,0.00
4,Afghanistan,AFG,2004,0.00
...,...,...,...,...
4020,Zimbabwe,ZWE,2018,82.01
4021,Zimbabwe,ZWE,2019,82.01
4022,Zimbabwe,ZWE,2020,82.01
4023,Zimbabwe,ZWE,2021,82.01


In [13]:
df = df[df['Year']==2000] 
pivot_df = df.pivot_table(index=['Entity', 'Code'], columns='Year', values='15.1.2 - Average proportion of Freshwater Key Biodiversity Areas (KBAs) covered by protected areas (%) - ER_PTD_FRHWTR').reset_index()
df_renamed = pivot_df.rename(columns={'Entity': 'country', 2000: 'avg_water_2000'})
merged_df = pd.merge(result_df, df_renamed, on='country', how='left')
result_df = merged_df.drop('Code', axis=1).dropna()

In [14]:
result_df

,country,avg_water_2001_2019,avg_water_2000
0,Algeria,6.194113,11.73
1,Angola,0.000000,42.83
2,Argentina,1.994157,27.99
3,Armenia,1.177986,24.98
4,Australia,2.911199,22.55
...,...,...,...
144,Venezuela,0.000000,71.73
145,Vietnam,7.847490,8.79
146,World,2.292080,28.12
147,Zambia,0.000000,56.84


In [15]:
###

In [ ]:
result_df.to_csv(f"{path_input}/channels21st/new_channels/water.csv", index=False)

